In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os 
import sys
import time
import tensorflow as tf
from tensorflow import keras

F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all,y_train_all),(x_test,y_test) = fashion_mnist.load_data()
x_valid,x_train = x_train_all[:5000],x_train_all[5000:]
y_valid,y_train = y_train_all[:5000],y_train_all[5000:]

In [3]:
#归一化操作  x = (s-u)/std   减去均值，除以方差

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#x_train:[none,28,28]-->[none,784]
x_train_scaler = scaler.fit_transform(
                            x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_valid_scaler = scaler.fit_transform(
                            x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)
x_test_scaler = scaler.fit_transform(
                            x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28,1)

In [5]:
#tf.keras.models.Sequential

model = keras.models.Sequential([
    
    keras.layers.Conv2D(
        filters=32,
        kernel_size=3,
        padding='same',
        activation='relu',
        input_shape=(28,28,1)),
    keras.layers.SeparableConv2D(
        filters=32,
        kernel_size=3,
        padding='same',
        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    
    keras.layers.SeparableConv2D(
        filters=64,
        kernel_size=3,
        padding='same',
        activation='relu',
        input_shape=(28,28,1)), 
    keras.layers.SeparableConv2D(
        filters=64,
        kernel_size=3,
        padding='same',
        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.SeparableConv2D(
        filters=128,
        kernel_size=3,
        padding='same',
        activation='relu',
        input_shape=(28,28,1)),
    keras.layers.SeparableConv2D(
        filters=128,
        kernel_size=3,
        padding='same',
        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Flatten(),
    
    keras.layers.Dense(128,activation='relu'),
    
    keras.layers.Dense(10,activation="softmax")])

model.compile(loss="sparse_categorical_crossentropy",
             optimizer="sgd",
             metrics=["accuracy"])


In [6]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
separable_conv2d (SeparableC (None, 28, 28, 32)        1344      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 14, 14, 64)        2400      
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 14, 14, 64)        4736      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 7, 7, 128)        

In [7]:
#TensorBoard, EarlyStopping, ModelCheckpoint
logdir = '.\cnn-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"fashion_mnist_model.h5")

history = model.fit(x_train_scaler,y_train,epochs=10,
                   validation_data=(x_valid_scaler,y_valid),
                   
                   callbacks = [
                                keras.callbacks.TensorBoard(logdir),
                                keras.callbacks.ModelCheckpoint(output_model_file,save_best_noly=True),
                                keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)]
                   )

W0916 15:58:36.986534  3396 deprecation.py:323] From F:\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 55000 samples, validate on 5000 samples
Epoch 1/10
55000/55000 [==============================] - 65s 1ms/sample - loss: 2.3027 - accuracy: 0.0982 - val_loss: 2.3028 - val_accuracy: 0.1012
Epoch 2/10
55000/55000 [==============================] - 69s 1ms/sample - loss: 2.3027 - accuracy: 0.0998 - val_loss: 2.3027 - val_accuracy: 0.0914
Epoch 3/10
55000/55000 [==============================] - 69s 1ms/sample - loss: 2.3027 - accuracy: 0.0988 - val_loss: 2.3027 - val_accuracy: 0.1024
Epoch 4/10
55000/55000 [==============================] - 69s 1ms/sample - loss: 2.3027 - accuracy: 0.0984 - val_loss: 2.3029 - val_accuracy: 0.0976
Epoch 5/10
55000/55000 [==============================] - 66s 1ms/sample - loss: 2.3027 - accuracy: 0.0999 - val_loss: 2.3030 - val_accuracy: 0.0986
Epoch 6/10
55000/55000 [==============================] - 56s 1ms/sample - loss: 2.3027 - accuracy: 0.0995 - val_loss: 2.3028 - val_accuracy: 0.1002


In [ ]:
def plt_learning_curve(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

plt_learning_curve(history)

In [ ]:
model.evaluate(x_test_scaler,y_test)